# Chaining Workflow

In [24]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from us_report_ext import (get_findings, 
                           load_split_md_docs, 
                           get_chroma_retrievers, 
                           retrieve_abnormal_docs,
                           get_prompt_template)

from us_report_ext._utils import read_markdown

In [25]:
llm_main = ChatOpenAI(model="gpt-3.5-turbo") # Main LLM for Prompt
llm_input = ChatOpenAI(model="gpt-3.5-turbo") # LLM for instruct input
embedding = OpenAIEmbeddings() # Chroma Embedding

## User Input

In [26]:
# User Text to Extract
user_text1 = """Generate US report with these findings:
- Mild fatty liver
- 2-mm left renal stone, 5-mm right renal cyst
- A 3-mm gallstone
"""

findings1 = get_findings(input_text=user_text1, llm = llm_input)
findings1

Findings(abnormal_liver=[Liver(finding='Mild fatty liver')], abnormal_kidney=[Kidney(finding='2-mm left renal stone'), Kidney(finding='5-mm right renal cyst')], abnormal_gallbladder=[GallBladder(finding='A 3-mm gallstone')])

## Abnormal Docs

In [27]:
md_header_splits_dict = load_split_md_docs("abnormal")
retriever_dict = get_chroma_retrievers(md_header_splits_dict, embedding= embedding)

abn_doc_dict1 = retrieve_abnormal_docs(retriever_dict, findings1)
abn_doc_dict1

{'abnormal_kidney': [Document(page_content='### Renal Stone  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s).  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s)\n```  \nExamples:  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A few non-obstructing caliceal stones at right kidney.  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- A few non-obstructing caliceal stones at right kidney.\n```', metadata={'Header 1': 'Kidney Findings', 'Header 3': 'Renal Stone'}),
  Document(page_content='### Renal Cortical Cyst(s)  \nHere is how to report renal cortical cyst according to Bosniak classification system.  \n#### Bosniak 1 (Simple Cyst)  \nUse this phase: "simple cortical cyst(s)" with <quantifier> as described in the english style guide.  \n```m

## Prompt 

In [10]:
prompt_temp1 = get_prompt_template(abn_doc_dict1)

## Chain

### Final Wrapper: `generate_report()`

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def generate_report(user_input: str,
                    llm_main = ChatOpenAI(model="gpt-3.5-turbo"), # Main LLM for Prompt
                    llm_input = ChatOpenAI(model="gpt-3.5-turbo"), # LLM for instruct input
                    embedding = OpenAIEmbeddings(), 
                    ) -> str:
    # Get findings
    findings = get_findings(input_text=user_input, llm = llm_input)
    # Dict of Markdown Splits 
    md_header_splits_dict = load_split_md_docs("abnormal")
    # Dict of retrievers
    retriever_dict = get_chroma_retrievers(md_header_splits_dict, embedding= embedding)
    # Dict of Abnormal Docs 
    abn_doc_dict = retrieve_abnormal_docs(retriever_dict, findings)
    # Prompt Template
    prompt_temp = get_prompt_template(abn_doc_dict)
    
    rag_chain = (
        {"user": RunnablePassthrough()}  |
        prompt_temp | 
        llm_main | 
        StrOutputParser()
    )
    
    report = rag_chain.invoke(user_input)
    return report
    

## Execute

In [21]:
report1 = generate_report("Generate normal US report")
print(report1)

**US OF THE UPPER ABDOMEN (MINI)**

**FINDINGS:**

**Liver:** Normal size and parenchymal echogenicity. No focal lesion.
**Gallbladder:** Well-distended gallbladder. No stone or mass.
**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. No stone, hydronephrosis or solid mass.

**IMPRESSION:**
- Normal liver parenchyma without focal lesion.


In [22]:
# User Text to Extract
user_text2 = """Generate US report with these findings:
- Mild fatty liver
- 6-mm left renal stone, 5-mm right renal cyst
"""

report2 = generate_report(user_text2)
print(report2)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size with mildly increased parenchymal echogenicity of the liver. No focal lesion.
**Gallbladder:** Well-distended gallbladder. No stone or mass.
**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A 6-mm non-obstructing caliceal stone at left kidney. A 5-mm simple right renal cyst. No hydronephrosis or solid mass.

**IMPRESSION:**
- Mild fatty liver without focal lesion.
- A 6-mm non-obstructing caliceal stone at left kidney.
- A 5-mm simple right renal cyst.


In [23]:
# User Text to Extract
user_text3 = """Generate US report with these findings:
- Severe fatty liver
- 6-mm left renal stone, 5-mm right renal cyst
- 2-cm gallstone
"""

report3 = generate_report(user_text3)
print(report3)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing impaired visualization of intrahepatic vasculature and right hemidiaphragm. No focal lesion.
**Gallbladder:** Distended gallbladder containing a 2-cm gallstone. No gallbladder wall thickening or pericholecystic fluid. No mass.
**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A 6-mm non-obstructing caliceal stone at left kidney. A 5-mm simple right renal cyst. No hydronephrosis or solid mass.

**IMPRESSION:**
- Severe fatty liver without focal lesion.
- A 6-mm non-obstructing caliceal stone at left kidney.
- A few simple right renal cysts, measuring up to 5 mm.
- A 2-cm gallstone without evidence of cholecystitis.
